# ARM的一些函数

- 将变量名转换成字符串
~~~ c
define name_to_str(name_31415926)  (#name_31415926) ~~~
- 直接调用name_to_str（变量名），函数将返回由‘变量名’转换成的字符串。

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

- 将字符串转换成整型
~~~ c
int atoi(const char *nptr); ~~~
- 该函数要求被转换的字符串是按十进制数理解的

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 字符串操作
#### 比较两个字符串
~~~ c
int strcmp(const char *s1, const char *s2); ~~~
- 返回值：若s1、s2字符串相等，则返回零；若s1大于s2，则返回大于零的数；否则，则返回小于零的数。
- 说明：strcmp()函数是根据ACSII码的值来比较两个字符串的；strcmp()函数首先将s1字符串的第一个字符值减去s2第一个字符，若差值为零则继续比较下去；若差值不为零，则返回差值。

[strspn, strcasestr, strstr](https://blog.csdn.net/hzraymond/article/details/8105494)


<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

#### 对内存空间进行操作的函数

- 将已开辟内存空间 s 的首 n 个字节的值设为值 c
~~~ c 
void *memset(void *s,int c,size_t n); ~~~
- memset()的深刻内涵：用来对一段内存空间全部设置为某个字符，一般用在对定义的字符串进行初始化为‘ ’或‘/0’；例:char a[100];memset(a, '/0', sizeof(a));
- 注意：memset是逐字节 拷贝的

- 内存拷贝，可以拷贝任何数据类型的对象，可以指定拷贝的数据长度
~~~ c
void *memcpy(void *dest, void *src, unsigned int count); ~~~
- 功能：由src所指内存区域复制count个字节到dest所指内存区域。

- 把src所指由NULL结束的字符串复制到dest所指的数组中
~~~c
char *strcpy(char *dest,char *src); ~~~
- 说明：src和dest所指内存区域不可以重叠且dest必须有足够的空间来容纳src的字符串。

- 注：一般用sizeof()函数来确定数据长度

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 阻塞性I/O操作
#### 简单休眠
- 静态方法定义并初始化一个等待列头：
~~~ c
DECLARE_WAIT_QUEUE_HEAD(name) ~~~
- 动态方法：
~~~ c 
wait_queue_head_t my_queue;
init_waitqueue_head(&myqueue); ~~~
- 休眠宏的调用
~~~ c
wait_event(queue, condition) //非中断休眠，通常不建议使用 通常与 wake_up(wait_queue_head_t *queue)唤醒函数配合使用
wait_event_interruptible(queue, condition) //通常使用wake_up——interruptible(wait_queue_head_t *queue)函数唤醒
wait_event_timeout(queue, condition, timeout) //后两个宏需要给定时间（jifft表示），当时间到期时，两个宏都会返回0，无论condition的值
wait_event_interruptible_timeout(queue, condition, timeout) ~~~

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### Linux进程上下文
<p> [linux进程上下文的理解](https://blog.csdn.net/liujiaoyage/article/details/31382321?utm_source=blogxgwz6)
- 所谓的“进程上下文”，可以看作是用户进程传递给内核的这些参数以及内核要保存的那一整套的变量和寄存器值和当时的环境等
- 所谓的“中断上下文”，其实也可以看作就是硬件传递过来的这些参数和内核需要保存的一些其他环境（主要是当前被打断执行的进程环境）
- CPU通过WSI软件中断切换内核态和用户态

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 分配内存
- linux内核把内存分为三个区段：可用于DMA的内存、常规内存、高端内存（大多数健全系统，所有内存都处于DMA内存，PCI设备无此限制）
- 内核负责管理系统物理内存，物理内存只能按页进行分配

#### malloc函数
[malloc()详解](https://www.cnblogs.com/ysys/p/6994091.html)

#### kmalloc 函数
~~~ c
void *kmalloc(size_t size,int flags) //第一个参数为要分配块的大小，第二个是分配标志：通常是GPF_KERNEL表示内存分配（最终总是调用get_free_pages来实现实际分配，这就是GFP_前缀的由来）
// flags还有很多标志：GFP_ATOMIC(不会休眠) GFP_USER ....等 ~~~
- size用来分配内存块的大小，如果申请任意数量的内存，那么得到的可能会多一些，最多会达到申请数量的两倍
#### get_free_pages 和相关函数
- 用来分配大块的内存
~~~ c
get_zeroed_page(unsigned int flags); //返回指向新页面的指针并将页面清零
__get_free_page(unsigned int flags); //返回指向新页面的指针不清零 可使用memset()函数清零
__get_free_pages(unsigned int flags, unsigned int order); //分配若干物理连续的页面，并返回指向该内存区域的第一个字节的指针，但不清零页面
//flags 与kmalloc中的类似，通常为GFP_KERNEL或GFP_ATOMIC。 order表示2^order个页
void free_page(unsigned long addr);
void free_page(unsigned long addr); // 释放页面 释放数目必须与开辟数目相同 ~~~
[__get_free_page 函数](http://blog.sina.com.cn/s/blog_5426448c0101bq4b.html)
- 分配内存并不是每次都能成功，需要对出错时进行处理，order值越大越容易失败，通常不超过5？？？？？
- 按页分配不会浪费内存，而使用kmalloc函数则会因分配粒度的原因浪费一定数量的内存

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 内存映射和DMA
#### 内存管理
- Linux是一个虚拟内存系统，用户程序使用的地址与硬件使用的物理地址是不等同的
- Linux地址类型列表
<p> [Linux虚拟地址空间布局](https://blog.csdn.net/wm_1991/article/details/52165574) 
    1. 用户虚拟地址（用户空间常规地址）
    2. 物理地址（处理器和系统内存使用）
    3. 总线地址（外围总线和内存使用）
    4. 内核逻辑地址（内核常规地址空间）-该地址映射了部分（或全部）内存，并经常被视为物理地址。kmalloc返回的即为逻辑地址）
    5. 内核虚拟地址（逻辑地址属于虚拟地址，但虚拟地址与物理地址的映射不必是一对一的，而这是逻辑地址的特点）
    
#### DMA
- 建立DMA环形缓冲区进行异步操作（适用于数据采集）
- 基于DMA的硬件使用总线地址，而非物理地址
- DMA映射建立了一个新的结构类型--dma_addr_t来表示总线地址
- 根据MDA缓冲区期望保留的时间长短，PCI代码区分两种类型的DMA映射:  
     1. 一致性DMA映射： 存在于驱动程序声明周期中。
      - 一致性映射必须保存在一致性缓存中，缓冲区必须同时被CPU和外围设备访问，开销很大
     2. 流式映射：通常为单独的操作建立 
      - 可以最大程度的优化性能，建议使用流式映射 。
       ~~~c
       dma_data_direction： //枚举类型定义了数据传输方向
           DMA_TO_DEVEIVE //数据由CPU发送到设备
           DMA_FROM_DEVICE //数据由设备发送到CPU
           DMA_BIDIRECTIONAL //双向移动
           DMA_NONE //调试  
           dma_addr_t dma_map_single(struct device *dev,void *buffer,size_t size,enum dma_data_direction direction);
           //返回值是总线地址
           //当传输完毕后，使用dma_unmap_single函数删除映射：
          void dma_unmap_single(struct device *dev,dma_addr_t dma_addr,size_t size,enum dma_data_direction direction);
           ~~~
- 流式DMA映射的几条原则:
   1. 缓冲区只能用于这样的传送，即其传送方向匹配于映射时给定的方向。
   2. 一旦缓冲区被映射，它将属于设备，而不是处理器,直到缓冲区被撤销映射前，驱动程序不能以任何方式访问其中的内容。
   3. 在DMA处于活动期间内，不能撤销对缓冲区映射，否则会严重破坏系统的稳定性。
- 驱动程序需要不经过撤销映射就访问流式DMA缓冲区的内容，有如下调用：
~~~ c 
void dma_sync_single_for_cpu(struct device *dev,dma_handle_t bus_addr,size_t size,enum dma_data_direction direction); ~~~
- 将缓冲区所有权交还给设备：
~~~ c
void dma_sync_single_for_device(struct device *dev,dma_handle_t bus_addr,size_t size,enum dma_data_direction direction);~~~

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### I/O内存分配和映射
- [ioremap() 函数解析](https://www.cnblogs.com/kazuki/p/9341414.html) 
~~~ c
//oremap宏定义在asm/io.h内：
#define ioremap(cookie,size)           __ioremap(cookie,size,0) ~~~
- 利用此函数将I/O内存映射到虚拟内存上，内核可以直接访问I/O内存资源（控制、状态、数据寄存器）

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 中断处理
- 驱动程序需要为自己设备的中断注册一个处理例程，并在中断到来时正确的处理，其中中断处理例程和其他代码并发运行
- 内核维护了一个中断信号线的注册表，在使用中断前需要申请一个中断通道或者中断请求IRQ，在使用完成后释放通道
####  <linux/sched.h>中的函数实现了请求和释放中断通道：
~~~ c
//申请中断
int request_irq(unsigned int irq, //申请的中断号
                irqreturn_t (*handler)(int, void *, struct pt_regs *), //中断处理函数指针
                unsigned long flags, //位掩码选项
                const char *dev_name, //传递给request_irq的字符串，用来在/proc/interrupts中显示终端的拥有者
                void *dev_id); //用于共享中断信号线
//当请求函数返回0时代表申请成功，负值则表示错误码
//释放中断
void free_irq(unsigned int irq, void *dev_id); ~~~
- flags 位掩码选项：
    #define IRQF_TRIGGER_NONE   0x00000000//IRQF表示中断请求Flags    
<p>#define IRQF_TRIGGER_RISING 0x00000001
<p>#define IRQF_TRIGGER_FALLING    0x00000002
<p>#define IRQF_TRIGGER_HIGH   0x000000#define IRQF_TRIGGER_LOW    0x00000008
<p>#define IRQF_TRIGGER_MASK   (IRQF_TRIGGER_HIGH | IRQF_TRIGGER_LOW | \IRQF_TRIGGER_RISING | IRQF_TRIGGER_FALLING)
<p>#define IRQF_TRIGGER_PROBE  0x00000010
<p>#define IRQF_DISABLED       0x00000020//调用此中断处理程序的时候禁止中断
<p>#define IRQF_SAMPLE_RANDOM  0x00000040//作为随机数的来源
<p>#define IRQF_SHARED     0x00000080//允许共享
<p>#define IRQF_PROBE_SHARED   0x00000100
<p>#define IRQF_TIMER      0x00000200//时钟中断
<p>#define IRQF_PERCPU     0x00000400//是SMP标志 
<p>#define SA_INTERRUPT        IRQF_DISABLED
<p>#define SA_SAMPLE_RANDOM    IRQF_SAMPLE_RANDOM
<p>#define SA_SHIRQ        IRQF_SHARED
<p>#define SA_PROBEIRQ     IRQF_PROBE_SHARED
<p>#define SA_PERCPU       IRQF_PERCPU
<p>#define SA_TRIGGER_LOW      IRQF_TRIGGER_LOW
<p>#define SA_TRIGGER_HIGH     IRQF_TRIGGER_HIGH
<p>#define SA_TRIGGER_FALLING  IRQF_TRIGGER_FALLING
<p>#define SA_TRIGGER_RISING   IRQF_TRIGGER_RISING
<p>#define SA_TRIGGER_MASK     IRQF_TRIGGER_MASK
<p>[Linux内核中断系统笔记](http://blog.sina.com.cn/s/blog_787fd82301013r60.html)
- 涉及I/O管脚中断设置：
<p>[GPIO及IRQ中断API函数](http://stevenysb.blog.163.com/blog/static/165402497201301053158958/)  
    [Linux GPIO中断实例](https://blog.csdn.net/huangkangying/article/details/78893643)
    


<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 有关Linux FILE一些介绍
[C语言FILE详解](https://blog.csdn.net/w371500241/article/details/53195377)
#### 文件的读写

1. 字符级别： fputc()、fgetc()

   函数原型: int fputc( int c, FILE *fp );

   功能：将字符 c 写入到 fp所指向的文件中。

   参数：c 的范围是 在unsigned char类型的范围(0-255)。

   返回值：运行成功返回该字符，运行失败，返回 EOF(-1)。

   函数原型：int fgetc(FILE *fp);

   功能：从fp所指向的文件爱你中读取一个字符存储在返回值中。

   参数：fp值，字符操作源文件。

   返回值：操作成功返回 该字符，操作失败(已经读到了文件文件末尾) 返回 “EOF”

2. fscanf()、fprintf()

   函数原型： int fprintf(FILE *stream, const char *format, ...);

   功能： 将 format格式对应的内容输出到 stream指向的文件中。

   举例：fprintf( stream, "%s%c", s, c ); 
   
   函数原型：int fscanf(FILE *stream, const char *format, ...);
   
   [printf, fprintf, sprintf,snprintf 区别](https://blog.csdn.net/shouwangzhe987/article/details/70216360)
   
   [puts()和gets()函数 用法 ()](https://www.cnblogs.com/wanhl/archive/2012/08/16/2641651.html)
3. line行级别： fputs()、fgets();
<p>[fgets函数原理初探](https://blog.csdn.net/chenglibin1988/article/details/8738070) 

4. lock块级别 fread()、fwrite()，可进行二进制文件的读写

~~~ c
size_t fread(void *ptr, size_t size_of_elements, size_t number_of_elements, FILE *a_file);
              
size_t fwrite(const void *ptr, size_t size_of_elements, size_t number_of_elements, FILE *a_file); ~~~

    功能:

    fread：从 a_file指向的文件中读取 size_of_elements  X  number_of_elements个数据，并存放到 ptr指向的内存当中。

    fwrite：从ptr指向的内存中读取 size_of_elements  X  number_of_elements个数据，存放到 a_file所指向的文件当中。

    参数说明：

    void * ptr表示的是你需要写入a_file文件中的数组名，或者其它结构的指针等等，

    size_of_elements 表示数组或者其他结构中每一个元素的大小（字节数）。

    number_of_elements表示一共有多少个元素  

    size_of_elements  X  number_of_elements为总的大小。 
#### rewind函数
- 文件指针FILE \*fp中，包含一个读写位置指针char \*_nextc，它指向下一次文件读写的位置。
- 每当进行一次读写后，该指针自动指向下一次读写的位置。当文件刚打开或创建时，该指针指向文件的开始位置。
- 可以用函数ftell()获得当前的位置指针，也可以用rewind()／fseek()函数改变位置指针，使其指向需要读写的位置。
- rewind()函数

  原型：void rewind(FILE *fp)

  作用：使文件fp的位置指针指向文件开始。
  
#### fseek函数和ftell函数文件随机访问
[fseek()的用法](https://blog.csdn.net/liupei815/article/details/80349091)

[fseek()与ftell()的工作原理](https://blog.csdn.net/libing403/article/details/73158972)

#### sscanf、scanf函数
- sscanf以固定字符串为输入源，按照一定的格式（选择匹配）输入到指定字符串中

[sscanf 的详细用法](https://www.cnblogs.com/listenerln/p/9186311.html)



<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 目录文件操作

[LINUX下历遍目录的方法](https://blog.csdn.net/dream_allday/article/details/75243818)

[Linux下DIR，dirent,stat等结构体详解](https://www.cnblogs.com/jikexianfeng/p/7084911.html)

[scandir, alphasort, versionsort - 为寻找项目扫描目录](https://www.cnblogs.com/ZXNblog/p/4067078.html)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 信号量
[信号量sem_init,sem_wait,sem_post](https://blog.csdn.net/u013457167/article/details/78318932)

[sem_wait sem_post信号量操作进本函数](https://blog.csdn.net/tietao/article/details/6825390)

[多线程pthread_create()函数](https://blog.csdn.net/wushuomin/article/details/80051295)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### mktie()函数
~~~ c
time_t mktime(strcut tm * timeptr); ~~~
- 函数说明：
  mktime()用来将参数timeptr所指的tm结构数据转换成从公元1970年1月1日0时0分0 秒算起至今的UTC时间所经过的秒数。
  
- 返回值:
  返回经过的秒数。
  
[mktime()函数使用](https://www.cnblogs.com/cyblogs/p/7616727.html)

### time函数和localtime获取系统时间和日期
[time函数和localtime获取系统时间和日期](https://blog.csdn.net/qq_22122811/article/details/52741483)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

~~~ c
struct tm 
{ 　
int tm_sec;		 /* 秒–取值区间为[0,59] */ 　　
int tm_min; 		 /* 分 - 取值区间为[0,59] */ 　　
int tm_hour; 	         /* 时 - 取值区间为[0,23] */ 　　
int tm_mday;		 /* 一个月中的日期 - 取值区间为[1,31] */ 　
int tm_mon;		 /* 月份（从一月开始，0代表一月） - 取值区间为[0,11] */ 
int tm_year; 	         /* 年份，其值从1900开始 */ 　
int tm_wday; 	         /* 星期–取值区间为[0,6]，其中0代表星期天，1代表星期一，以此类推 */ 　
int tm_yday; 	         /* 从每年的1月1日开始的天数–取值区间为[0,365]，其中0代表1月1日，1代表1月2日，以此类推 */ 　
int tm_isdst; 	         /* 夏令时标识符，实行夏令时的时候，tm_isdst为正。不实行夏令时的进候，tm_isdst为0；不了解情况时，tm_isdst()为负。*/ 　
long int tm_gmtoff;	 /*指定了日期变更线东面时区中UTC东部时区正秒数或UTC西部时区的负秒数*/ 　　
const char *tm_zone;     /*当前时区的名字(与环境变量TZ有关)*/ 　
};  ~~~


<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

In [9]:
from ipywidgets import widgets
file = open("linux.jpg", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='jpg',
    width=800,
    height=1000,
)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x01J\x08\x06\x00\x00\x00\xc1\x90!\xd1\x…

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### linux 删除文件夹命令：

- 使用rm -rf 目录名字 命令即可

- -r 就是向下递归，不管有多少级目录，一并删除
- -f 就是直接强行删除，不作任何提示的意思

- eg

- 删除文件夹实例：rm -rf /var/log/httpd/access
- 将会删除/var/log/httpd/access目录以及其下所有文件、文件夹

- 删除文件使用实例：rm -f /var/log/httpd/access.log
- 将会强制删除/var/log/httpd/access.log这个文件

### linux命令-date显示或者设置时间
[date 设置系统时间](https://blog.csdn.net/sodjfowehyrtf9ehnifj/article/details/52469160)


<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### linux中system()函数：
- 头文件 
 ＃include 
- 定义函数 
 int system(const char * string); 
- system()会调用fork()产生子进程，由子进程来调用/bin/sh-c string来执行参数string字符串所代表的命令，此命令执行完后随即返回原调用的进程。在调用system()期间SIGCHLD 信号会被暂时搁置，SIGINT和SIGQUIT 信号则会被忽略。 返回值 =-1:出现错误 =0:调用成功，但是没有出现子进程。 >0:成功退出的子进程的id。 如果system()在调用/bin/sh时失败则返回127，其他失败原因返回-1。若参数string为空指针(NULL)，则返回非零值>。如果system()调用成功则最后会返回执行shell命令后的返回值，但是此返回值也有可能为 system()调用/bin/sh失败所返回的127，因此最好能再检查errno 来确认执行成功。
- [system()](https://www.cnblogs.com/tdyizhen1314/p/4902560.html)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### Linux 进程间通信
- 共享内存 shmget()、shmat()、shmdt()、shmctl()
- [共享内存详解](https://www.cnblogs.com/52php/p/5861372.html)
- [shmget](http://www.blogjava.net/sunzhong/articles/297435.html)



<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

![jupyter](./linux.jpg)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

### 驱动命令的应用

- [Linux下getopt()函数的简单使用](http://www.cnblogs.com/qingergege/p/5914218.html)

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>

<font color=#ff0000 size=10> ----------------------------------------------------------- </font>